# Malicious Network Traffic Classification Using Deep Neural Networks

### CSEC620 - Project 1 - Mehul Sen

This project uses a deep neural network to classify network traffic into benign and malicious categories. The model can detect various types of attacks, such as denial of service, distributed denial of service, brute force, botnet, web attacks, and port scans. It is trained on a dataset labeled CICIDS2017, which contains network traffic data captured over five days. The project demonstrates how deep learning can detect and classify malicious network traffic.

## Import Libraries
- Import common data science and machine learning libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pickle

from tensorflow import keras
from keras import layers
from keras import Sequential
from keras.utils import to_categorical
from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout
from keras.optimizers import SGD


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

## Import and Preprocess the Dataset
- Load and concatenate raw datasets into one DataFrame
- Do data cleaning and preprocessing 
- Save preprocessed dataset to file

In [ ]:
file_path = 'preprocessed_network_dataset.csv'  # Replace with the path to your file

# Check if the file exists
if not os.path.exists(file_path):
    print("[+]\tConcatenating Datasets into one...")
    df_monday = pd.read_csv('Dataset\Monday-WorkingHours.pcap_ISCX.csv')
    df_tuesday = pd.read_csv('Dataset\Tuesday-WorkingHours.pcap_ISCX.csv')
    df_wednesday = pd.read_csv('Dataset\Wednesday-workingHours.pcap_ISCX.csv')
    df_thursday_morning = pd.read_csv('Dataset\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
    df_thursday_afternoon = pd.read_csv('Dataset\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
    df_friday_morning = pd.read_csv('Dataset\Friday-WorkingHours-Morning.pcap_ISCX.csv')
    df_friday_afternoon_1 = pd.read_csv('Dataset\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
    df_friday_afternoon_2 = pd.read_csv('Dataset\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
    df = pd.concat([df_monday, df_tuesday, df_wednesday, df_thursday_morning, df_thursday_afternoon, df_friday_morning, df_friday_afternoon_1, df_friday_afternoon_2])
    print("[+]\tCompleted concatenation.")

    print("[+]\tPreprocessing Dataset...")
    # Fix Column title error within the dataset
    df.rename(columns=lambda x: x.strip(), inplace=True)
    # Remove non numerical columns
    column_data_types = df.dtypes
    for column_name, data_type in column_data_types.items():
        if not pd.api.types.is_numeric_dtype(data_type):
            if column_name != 'Label':
                df = df.drop(columns=[column_name])
    # Remove attacks that have only a small number of examples available
    # THRESHOLD - Minimum number of values required in dataset
    threshold = 100
    print("[+]\t\tLabel Categories: \n",df['Label'].value_counts())
    value_counts = df['Label'].value_counts()
    valid_values = value_counts[value_counts >= threshold].index.tolist()
    df = df[df['Label'].isin(valid_values)]
    # Categorize each attack type into more generalized categories
    attack_group = {
        'BENIGN': 'benign',
        'Bot': 'botnet',
        'DDoS': 'ddos',
        'DoS GoldenEye': 'dos',
        'DoS Hulk': 'dos',
        'DoS Slowhttptest': 'dos',
        'DoS slowloris': 'dos',
        'FTP-Patator': 'bruteforce',
        'PortScan': 'portscan',
        'SSH-Patator': 'bruteforce',
        'Web Attack � Brute Force': 'webattack',
        'Web Attack � XSS': 'webattack'
    }
    df['Label'] = df['Label'].replace(attack_group)
    # Clean data with missing values and columns that have the same values throughout the dataset
    unchanging_columns = [col for col in df.columns if df[col].nunique() == 1]
    df.drop(unchanging_columns, axis=1, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    print("[+]\tCompleted preprocessing dataset.")
    # Save or Load dataset file
    print("[+]\tSaving preprocessed dataset...")
    df.to_csv(file_path, index=False)
    print("[+]\tCompleted saving dataset.")
else:
    print("[+]\tRetrieving preprocessed dataset: ", file_path)
    df = pd.read_csv(file_path)
    print("[+]\tCompleted retreiving dataset.")
print("[+]\t\tNew Label Categories: \n",df['Label'].value_counts())
print("[+]\t\tDataset Shape: ",df.shape)

## Split Data into Training and Testing Data
- Encode labels
- Split features and labels
- Normalize features
- Split into train/test sets

In [ ]:
# Encode Labels
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# Define your features (X) and target (y)
X = df.drop(columns=['Label'])
Y = df['Label']

# Normalize the dataset
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into training (70%) and test (30%) sets
test_percentage = 0.3
X_train, X_test, Y_train, Y_test = train_test_split(X_normalized, Y, test_size=test_percentage, random_state=42)

# Print the shapes of the resulting sets to verify
print("[+]\t\tX_train shape: ",X_train.shape)
print("[+]\t\tY_train shape: ",Y_train.shape)
print("[+]\t\tX_test shape: ",X_test.shape)
print("[+]\t\tY_test shape: ",Y_test.shape)

# Calculate features and number of classes
features = X_train.shape[1]
num_classes = len(le.classes_)
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

print("[+]\t\tFeatures: ", features)
print("[+]\t\tClasses: ", num_classes)

## Training and Testing Model
- Define model architecture
- Compile Model
- Train model on training data
- Save/Load trained model
- Evaluate Model accuracy on test set

In [ ]:
# Path to store the model, replace with '.\Model\pretrained_malicious_traffic_classification_model.pkl to run the pretrained model
#model_path = 'malicious_traffic_classification_model.pkl'
model_path = '.\Model\pretrained_malicious_traffic_classification_model.pkl'

# Check if the file exists
if not os.path.exists(model_path):
    print("[+]\tTraining Model...")
    
    # Define and compile model
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, input_shape=(features, 1), padding="same", activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, Y_train, epochs=15, batch_size=50)
    print("[+]\tCompleted training model.")
    # Save the model to file
    print("[+]\tSaving trained model...")
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)
    print("[+]\tCompleted saving model.")
else:
    # Load model from file
    print("[+]\tRetrieving pretrained model: ", model_path)
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    print("[+]\tCompleted retreiving model.")
print("[+]\tTesting model accuracy...")
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print("[+]\t\tModel Accuracy: ",test_accuracy)

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train'], loc='upper left')

plt.tight_layout()
plt.show()

## Performance
- Calculate Precision, Recall, and F1 score for the model.

In [ ]:
# Make predictions on the test data
Y_pred = model.predict(X_test)

# Convert the predicted probabilities back to class labels
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Convert one-hot encoded true labels back to class labels
Y_test_classes = np.argmax(Y_test, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(Y_test_classes, Y_pred_classes, average='weighted')
recall = recall_score(Y_test_classes, Y_pred_classes, average='weighted')
f1 = f1_score(Y_test_classes, Y_pred_classes, average='weighted')

print("[+]\t\tPrecision: ", precision)
print("[+]\t\tRecall: ", recall)
print("[+]\t\tF1 Score: ", f1)

## Cleanup
- Delete the concatenated dataset `preprocessed_network_dataset.csv`
- Delete the trained model `malicious_traffic_classification_model.pkl`

In [ ]:
print("[+]\tCleaning up...")
try:
    os.remove("preprocessed_network_dataset.csv")
    print("[+]\t\tConcatenated Dataset has been deleted.")
except FileNotFoundError:
    print("[-]\t\tConcatenated Dataset not found.")
except Exception as e:
    print("[-]\t\t An error occurred: ", e)

try:
    os.remove("malicious_traffic_classification_model.pkl")
    print("[+]\t\tTrained Model has been deleted.")
except FileNotFoundError:
    print("[-]\t\tTrained Model not found.")
except Exception as e:
    print("[-]\t\t An error occurred: ", e)
print("[+]\tClean up completed.")